# Imports e definições gerais

In [123]:
import pandas as pd
from dash import Dash, html, dcc, dash_table, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go

In [124]:
ARQUIVO_CSV = 'dialise_fav_final3.csv'
SEPARADOR   = ';'

df = pd.read_csv(ARQUIVO_CSV, sep=SEPARADOR)
df.sample(5)

C:\Users\Marnie\AppData\Local\Temp\ipykernel_16284\3187438065.py:4: DtypeWarning:

Columns (20,37,104) have mixed types. Specify dtype option on import or set low_memory=False.



,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,fav_ACF_FLEBIT,fav_ACF_HEMATO,fav_ACF_VEIAVI,fav_ACF_PULSO,fav_ACF_VEIDIA,fav_ACF_ARTDIA,fav_ACF_FREMIT,fav_AP_NATJUR,fav_repeticao,espera_fav
336348,201905,PG,430770,2232030,4319200174586,2019-05-01,305010107.0,2578.61,430770,5,...,N,N,S,S,0,0,3.0,NaN,46.0,NaN
305412,201812,EP,430000,2242400,4318200927546,2018-12-01,305010107.0,2596.42,431690,39,...,N,N,S,S,0,0,3.0,2062.0,62.0,NaN
684927,202401,PG,430460,2700662,4323202089822,2024-01-01,305010107.0,3373.58,430460,36,...,N,N,S,S,0,0,3.0,2062.0,32.0,NaN
735888,202408,PG,431440,2253054,4324200103069,2024-08-01,305010107.0,3318.65,431440,5,...,N,N,S,S,0,0,3.0,3999.0,49.0,NaN
590651,202210,EP,430000,2707829,4322213614873,2022-10-01,305010107.0,1746.86,432150,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [125]:
paleta = {
    'azul_principal': '#0077B6',
    'azul_claro'    : '#90E0EF',
    'ciano'         : '#48CAE4',
    'azul_escuro'   : '#023E8A',
    'cinza_texto'   : '#6C757D',
    'fundo'         : '#FFFFFF',
    'cinza_claro'   : '#F8F9FA'
}

# Tratamento de Dados

In [126]:
df_espera = pd.DataFrame()

# Pergunta 1 e 2
df_espera['data_primeira_dialise']      = df['ATD_DTPDR']
df_espera['cronico']                    = df['Cronico']
df_espera['data_atendimento_dialitico'] = df['AP_CMP']
df_espera['obito']                      = df['AP_OBITO']
df_espera['data_atendimento_fav']       = df['fav_AP_CMP']
df_espera.sample(10)

# Pergunta 3
df_espera['genero'] = df['AP_SEXO']
df_espera['idade'] = df['AP_NUIDADE']

# Pergunta 4
df_espera['hospital'] = df['fav_AP_CODUNI']

In [127]:
# Para os casos de morte, podemos desconsiderar o tempo de espera (?)
df_espera_mortos_validos = df_espera[df_espera.obito == 1][df_espera.data_atendimento_fav.notna()]

# Remover todos os mortos
df_espera = df_espera[df_espera.obito != 1]

# Inserir apenas falecidos válidos
df_espera = pd.concat([df_espera, df_espera_mortos_validos])
df_espera.sample(10)

C:\Users\Marnie\AppData\Local\Temp\ipykernel_16284\2896329900.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital
345852,2017-04-01,repetido,2019-07-01,0,2020-03-01,M,73,2707829.0
562745,2013-09-06,repetido,2022-06-01,0,NaN,F,64,NaN
250081,2016-12-02,repetido,2018-03-01,0,2017-08-01,F,64,2237253.0
181037,2016-02-04,repetido,2017-03-01,0,2016-01-01,F,57,2232030.0
124150,2015-07-07,repetido,2016-05-01,0,NaN,M,85,NaN
670011,2023-11-08,agudo,2023-11-01,0,NaN,M,62,NaN
376664,2019-07-11,repetido,2019-12-01,0,2019-10-01,F,64,2231069.0
447635,2009-05-04,repetido,2020-11-01,0,2015-08-01,M,42,2257815.0
609715,2020-12-23,repetido,2023-01-01,0,NaN,F,89,NaN
459275,2011-05-19,repetido,2021-01-01,0,NaN,F,55,NaN


In [128]:
# Priorizar remover repetidos NaN
df_espera = df_espera[df_espera.cronico != 'repetido']
df_espera.sample(5)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital
609511,2022-10-26,cronico,2023-01-01,0,2024-04-01,M,49,2262509.0
668068,2023-07-01,cronico,2023-10-01,0,2023-02-01,M,57,2254611.0
42468,2015-02-26,agudo,2015-03-01,0,NaN,M,57,NaN
750983,2024-09-23,agudo,2024-11-01,0,2021-10-01,M,68,2232022.0
460111,2021-01-11,agudo,2021-01-01,0,NaN,M,53,NaN


In [129]:
# Remover observações que não são agudos e que não tiveram atendimento FAV
# Para garantir, vamos remover todos que não tiveram data_primeira_dialise também
df_espera = df_espera[df_espera.cronico != 'agudo']
df_espera = df_espera[df_espera.data_atendimento_fav.notna()]
df_espera = df_espera[df_espera.data_primeira_dialise.notna()]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital
449230,2020-09-21,cronico,2020-12-01,0,2021-01-01,F,26,2242397.0
515616,2021-07-24,cronico,2021-10-01,0,2022-03-01,F,63,2261057.0
512448,2021-07-16,cronico,2021-10-01,0,2021-02-01,F,50,2262274.0
257623,2018-01-19,cronico,2018-04-01,0,2018-05-01,M,66,2231778.0
76859,2015-06-25,cronico,2015-09-01,0,2015-08-01,M,73,2248204.0
453485,2020-09-06,cronico,2020-12-01,0,2020-09-01,M,60,2237253.0
476730,2021-01-04,cronico,2021-04-01,0,2020-02-01,M,42,2701146.0
595496,2022-08-19,cronico,2022-11-01,0,2022-09-01,M,71,2701146.0
227447,2000-11-22,cronico,2017-11-01,0,2017-12-01,F,45,2701146.0
714402,2024-02-12,cronico,2024-05-01,0,2024-06-01,M,62,2242400.0


In [130]:
# Calcular tempo de espera
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Calcular a diferença
df_espera['tempo_entre_fav_e_primeira_dialise'] = (
    df_espera['data_atendimento_fav'] - df_espera['data_primeira_dialise']
).dt.days  # em dias

df_espera['tempo_anos'] = df_espera['tempo_entre_fav_e_primeira_dialise'] / 365.25

# Remover esperas negativas
df_espera = df_espera[df_espera['tempo_entre_fav_e_primeira_dialise'] > 0]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital,tempo_entre_fav_e_primeira_dialise,tempo_anos
45158,2014-12-19,cronico,2015-03-01,0,2015-01-01,M,44,2255456.0,13,0.035592
62707,2015-03-13,cronico,2015-06-01,0,2015-05-01,M,48,2241021.0,49,0.134155
678166,2023-09-13,cronico,2023-12-01,0,2024-03-01,M,65,2261898.0,170,0.465435
5589,2013-10-02,cronico,2014-08-01,0,2015-09-01,F,50,2701146.0,699,1.913758
42567,2014-12-26,cronico,2015-03-01,0,2015-01-01,F,47,2247488.0,6,0.016427
1730,2008-10-16,cronico,2014-08-01,0,2014-09-01,F,46,2252295.0,2146,5.875428
429363,2020-05-18,cronico,2020-08-01,0,2020-06-01,F,33,2230577.0,14,0.038330
129074,2016-03-02,cronico,2016-06-01,0,2016-04-01,F,82,2231069.0,30,0.082136
678748,2023-09-13,cronico,2023-12-01,0,2023-12-01,F,68,2228602.0,79,0.216290
557992,2022-02-15,cronico,2022-05-01,0,2022-04-01,F,54,2254611.0,45,0.123203


In [131]:
# Manter dados apenas a partir de janeiro de 2014 devido criação do dataset
df_espera['data_atendimento_dialitico'] = pd.to_datetime(df_espera['data_atendimento_dialitico'])
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

df_espera = df_espera[
        (df_espera['data_atendimento_dialitico'] >= '2014-01-01') &
        (df_espera['data_atendimento_fav'] >= '2014-01-01') &
        (df_espera['data_primeira_dialise'] >= '2014-01-01') ]

In [132]:
df_perfil = df_espera[(df_espera.genero.notna()) & (df_espera.idade.notna())]

df_perfil['perfil_paciente'] = pd.cut(
    df_perfil['idade'],
    bins=[0, 18, 40, 60, 150],
    labels=['Jovem', 'Adulto Jovem', 'Meia Idade', 'Idoso']
)
df_perfil['ano_primeira_dialise'] = pd.to_datetime(df_perfil['data_primeira_dialise']).dt.year

In [133]:
df_hospital = df_espera[df_espera.hospital.notna()]
df_hospital.sample()
df_hospital['hospital'] = df_hospital['hospital'].astype(str)

df_hospital['ano_primeira_dialise'] = pd.to_datetime(df_hospital['data_primeira_dialise']).dt.year
# Criar coluna de ano
df_hospital['ano_primeira_dialise'] = pd.to_datetime(df_hospital['data_primeira_dialise']).dt.year

# 1) Após o início da diálise crônica, qual é o tempo médio até a confecção da Fístula Arterial Venosa (FAV) nos hospitais de POA/RS?

In [134]:
# Gráfico de tempo de espera
max_tempo = df_espera['tempo_entre_fav_e_primeira_dialise'].max()
bins = list(range(0, int(max_tempo) + 10, 10))

# Plotar histograma
fig1_p1 = px.histogram(
    df_espera,
    x='tempo_entre_fav_e_primeira_dialise',
    nbins=len(bins),
    title='Distribuição do Tempo de Espera para Atendimento no FAV',
    labels={
        'tempo_entre_fav_e_primeira_dialise': 'Tempo de espera (dias)',
        'count': 'Quantidade de pessoas'
    },
    color_discrete_sequence=[paleta['azul_principal']]  # Cor das barras
)

# Layout personalizado com a paleta
fig1_p1.update_layout(
    xaxis_title='Tempo de espera (dias)',
    yaxis_title='Quantidade de pessoas',
    bargap=0.05,
    template='plotly_white',
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    xaxis=dict(
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    ),
    yaxis=dict(
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    )
)

fig1_p1.show()

In [135]:
df_espera['ano_primeira_dialise'] = df_espera['data_primeira_dialise'].dt.year
df_agrupado = df_espera.groupby('ano_primeira_dialise')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

fig2_p1 = px.bar(
    df_agrupado,
    x='ano_primeira_dialise',
    y='tempo_entre_fav_e_primeira_dialise',
    text='tempo_entre_fav_e_primeira_dialise',
    labels={
        'ano_primeira_dialise': 'Ano da Primeira Diálise',
        'tempo_entre_fav_e_primeira_dialise': 'Tempo médio de espera (dias)'
    },
    title='Tempo Médio de Espera por Ano da Primeira Diálise até a FAV',
    color='tempo_entre_fav_e_primeira_dialise',
    color_continuous_scale='Blues'
)

# Personalizar layout
fig2_p1.update_traces(texttemplate='%{text:.0f} dias', textposition='outside')

fig2_p1.update_layout(
    xaxis=dict(tickmode='linear'),
    yaxis_title='Tempo médio de espera (dias)',
    xaxis_title='Ano da primeira diálise',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor='white',
    width=900,
    height=500
)

fig2_p1.show()


# 2) Houve mudanças na demora ao longo dos anos, especialmente em períodos críticos como a pandemia de COVID-19?

In [136]:
# Garantir que a coluna data_primeira_dialise está em datetime
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Criar coluna de ano-mês
df_espera['ano_mes'] = df_espera['data_primeira_dialise'].dt.to_period('M').astype(str)

# Agrupar por ano-mês
df_mes = df_espera.groupby('ano_mes')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

# Converter ano-mês para datetime (para plotar no eixo)
df_mes['ano_mes_dt'] = pd.to_datetime(df_mes['ano_mes'])

# Gráfico de Linha (sem pontos)
fig1_p2 = go.Figure()

fig1_p2.add_trace(go.Scatter(
    x=df_mes['ano_mes_dt'],
    y=df_mes['tempo_entre_fav_e_primeira_dialise'],
    mode='lines',
    name='Tempo médio de espera',
    line=dict(color=paleta['azul_principal'], width=3)
))

# Tarja vermelha para COVID-19 (março de 2020 até dezembro de 2021)
fig1_p2.add_vrect(
    x0="2020-03-01", x1="2021-12-31",
    fillcolor="red", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="COVID-19", annotation_position="top left",
    annotation_font_size=12, annotation_font_color="red"
)

# Layout final com paleta aplicada
fig1_p2.update_layout(
    title='Média do Tempo de Espera por Mês',
    xaxis_title='Ano',
    yaxis_title='Tempo médio de espera (dias)',
    template='plotly_white',
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    width=1000,
    height=500,
    xaxis=dict(
        tickformat='%Y',
        dtick="M12",
        showgrid=True,
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    )
)

fig1_p2.show()

In [137]:
# Preparar dados
df_evolucao = df_espera.groupby("ano_primeira_dialise")['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

# Criar o gráfico com melhorias
fig2_p2 = px.line(
    df_evolucao,
    x="ano_primeira_dialise",
    y="tempo_entre_fav_e_primeira_dialise",
    markers=True,
    title="Evolução do Tempo Médio de Espera entre FAV e Primeira Diálise"
)

# Melhorias visuais
fig2_p2.update_traces(line=dict(width=3), marker=dict(size=8))

fig2_p2.update_layout(
    title={
        'text': 'Evolução do Tempo Médio de Espera entre FAV e Primeira Diálise',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Ano da Primeira Diálise",
    yaxis_title="Tempo Médio de Espera (dias)",
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="#333333"
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        gridcolor='lightgrey',
        tickmode='linear'
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        gridcolor='lightgrey'
    ),
    hovermode="x unified"
)

fig2_p2.show()

# 3) Existem perfis de pacientes que apresentam maior demora para a confecção da FAV?

In [147]:
genero_cores = {
    'F': paleta['azul_escuro'],
    'M': paleta['azul_claro']
}

In [148]:
fig1_p3 = px.scatter(
    df_perfil,
    x='idade',
    y='tempo_entre_fav_e_primeira_dialise',
    color='genero',
    labels={
        'idade': 'Idade',
        'tempo_entre_fav_e_primeira_dialise': 'Tempo de espera (dias)',
        'genero': 'Gênero'
    },
    title='Perfil dos Pacientes vs Tempo de Espera para Confecção da FAV',
    color_discrete_map=genero_cores
)

# Layout com paleta médica
fig1_p3.update_layout(
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    xaxis=dict(
        title='Idade',
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto']
    ),
    yaxis=dict(
        title='Tempo de espera (dias)',
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto']
    )
)

fig1_p3.show()

In [150]:
# Agrupar dados por perfil e gênero
df_perfil_group = (
    df_perfil.groupby(['perfil_paciente', 'genero'])['tempo_entre_fav_e_primeira_dialise']
    .mean()
    .reset_index()
)

# Criar o gráfico de barras empilhadas
fig2_p3 = px.bar(
    df_perfil_group,
    x="perfil_paciente",
    y="tempo_entre_fav_e_primeira_dialise",
    color="genero",
    color_discrete_map=genero_cores,
    barmode="stack",
    text_auto=".1f",
    title="Tempo Médio de Espera entre FAV e Primeira Diálise por Perfil de Paciente e Gênero"
)

# Melhorias visuais e layout
fig2_p3.update_layout(
    title={
        'text': 'Tempo Médio de Espera por Perfil de Paciente e Gênero',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Perfil do Paciente",
    yaxis_title="Tempo Médio de Espera (dias)",
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color=paleta['cinza_texto']
    ),
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        gridcolor='lightgrey'
    ),
    legend_title_text='Gênero',
    bargap=0.15
)

# Ajuste de rótulos nas barras
fig2_p3.update_traces(textposition='outside')

fig2_p3.show()

C:\Users\Marnie\AppData\Local\Temp\ipykernel_16284\1712906381.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# 4) Existe variação no tempo de espera para a confecção da FAV entre diferentes unidades hospitalares de Porto Alegre?

In [140]:
df_espera['hospital'] = df_espera['hospital'].astype(str)

top_25_maiores = df_espera.sort_values(by='tempo_entre_fav_e_primeira_dialise', ascending=False).head(25)

top_25_menores = df_espera.sort_values(by='tempo_entre_fav_e_primeira_dialise', ascending=True).head(25)

df_extremos = pd.concat([top_25_maiores, top_25_menores]).reset_index(drop=True)
df_extremos[['hospital', 'idade', 'genero', 'tempo_entre_fav_e_primeira_dialise', 'data_primeira_dialise', 'data_atendimento_fav']]
df_extremos.sample(5)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital,tempo_entre_fav_e_primeira_dialise,tempo_anos,ano_primeira_dialise,ano_mes
15,2014-08-08,cronico,2022-10-01,0,2023-06-01,F,38,2262460.0,3219,8.813142,2014,2014-08
41,2019-11-30,cronico,2020-02-01,0,2019-12-01,M,52,2247984.0,1,0.002738,2019,2019-11
38,2018-08-31,cronico,2018-11-01,0,2018-09-01,M,72,2237571.0,1,0.002738,2018,2018-08
9,2014-08-16,cronico,2018-02-01,0,2024-02-01,M,45,2263858.0,3456,9.462012,2014,2014-08
37,2018-07-31,cronico,2018-10-01,0,2018-08-01,M,70,2257815.0,1,0.002738,2018,2018-07


In [141]:
# Agrupar por hospital e ano
df_heat = df_hospital.groupby(['hospital', 'ano_primeira_dialise'])['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

# Pivotar para criar matriz do heatmap
heatmap_data = df_heat.pivot(
    index='hospital',
    columns='ano_primeira_dialise',
    values='tempo_entre_fav_e_primeira_dialise'
)

# 🔵 Substituir NaNs por -1 (para representar ausência de dados no heatmap)
z_data = heatmap_data.fillna(-1).values

# 🔵 Criar texto do hover personalizado
hover_text = []
for i, hospital in enumerate(heatmap_data.index):
    row = []
    for j, ano in enumerate(heatmap_data.columns):
        valor = z_data[i][j]
        if valor == -1:
            row.append(
                f"Hospital: {hospital}<br>Ano: {ano}<br><b>Sem informação</b>"
            )
        else:
            row.append(
                f"Hospital: {hospital}<br>Ano: {ano}<br>Tempo médio: {valor:.0f} dias"
            )
    hover_text.append(row)

# 🔵 Definir uma escala de cores personalizada com preto para NaN (-1)
colorscale = [
    [0.0, 'rgb(99,99,99)'],                           # NaN → preto
    [0.00001, 'rgb(239,243,255)'],             # início da paleta Blues
    [0.2, 'rgb(189,215,231)'],
    [0.4, 'rgb(107,174,214)'],
    [0.6, 'rgb(33,113,181)'],
    [1.0, 'rgb(8,48,107)']                     # fim da paleta Blues
]

# 🔵 Construir o Heatmap
fig1_p4 = go.Figure(
    data=go.Heatmap(
        z=z_data,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        text=hover_text,
        hoverinfo='text',
        colorscale=colorscale,
        colorbar=dict(title='Tempo médio (dias)'),
        zmin=heatmap_data.min().min(),
        zmax=heatmap_data.max().max(),
        hoverongaps=False
    )
)

# 🔵 Layout
fig1_p4.update_layout(
    title='Tempo Médio de Espera para FAV - Hospital x Ano',
    xaxis_title='Ano da Primeira Diálise',
    yaxis_title='Hospital',
    xaxis=dict(side='bottom'),
    yaxis=dict(autorange="reversed"),
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
)

fig1_p4.show()

# DASH Dashboard

In [142]:
NOMES = 'Gabriela Dellamora, Luciano da Rocha, Nícolas Moraes, Tiago Gambogi'

In [160]:
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.title = "Dashboard FAV"

# ✅ Função para criar cards de gráfico
def gerar_card(titulo, grafico):
    return dbc.Card(
        [
            dbc.CardHeader(html.H5(titulo, className="text-center fw-bold")),
            dbc.CardBody(
                dcc.Graph(figure=grafico, config={'displayModeBar': False}),
                style={"height": "100%"}
            )
        ],
        className="shadow-sm mb-4 h-100"
    )

# ✅ Função para criar cards de KPI
def gerar_kpi(titulo, valor, cor="primary"):
    return dbc.Card(
        dbc.CardBody([
            html.H6(titulo, className="card-title"),
            html.H4(valor, className="card-text fw-bold")
        ]),
        color=cor, inverse=True, className="shadow-sm mb-4"
    )

# ✅ Layout do App
app.layout = dbc.Container([

    # ===== HEADER =====
    dbc.Row([
        dbc.Col(html.H1(
            "Dashboard - Tempo de Espera na FAV",
            className="text-center text-primary my-4"
        ))
    ]),
    dbc.Row([
        dbc.Col(html.H5(
            "Análise dos tempos de espera entre a primeira diálise e o atendimento na FAV.",
            className="text-center text-secondary mb-4"
        ))
    ]),

    # ===== KPIs =====
    dbc.Row([
        dbc.Col(gerar_kpi("Tempo Médio de Espera", f"{round(df_espera['tempo_entre_fav_e_primeira_dialise'].mean(), 1)} dias", "primary"), width=3),
        dbc.Col(gerar_kpi("Total de Hospitais", df_espera['hospital'].nunique(), "info"), width=3),
        dbc.Col(gerar_kpi("Total de Pacientes", df_espera.shape[0], "success"), width=3),
        dbc.Col(gerar_kpi("Período da Base", f"{df_espera['ano_primeira_dialise'].min()} - {df_espera['ano_primeira_dialise'].max()}", "warning"), width=3),
    ], class_name="mb-4"),

    # ===== TABS =====
    dbc.Tabs([

        # 🔵 Aba 1: Visão Geral
        dbc.Tab(label="Visão Geral", tab_id="tab-geral", children=[
            dbc.Row([
                dbc.Col(gerar_card("Distribuição do Tempo de Espera", fig1_p1), width=6),
                dbc.Col(gerar_card("Tempo Médio por Ano", fig2_p1), width=6),
            ]),
        ]),

        # 🔵 Aba 2: Análises Detalhadas
        dbc.Tab(label="Análises Detalhadas", tab_id="tab-detalhes", children=[
            dbc.Row([
                dbc.Col(gerar_card("Dispersão: Idade x Tempo de Espera", fig1_p3), width=6),
                dbc.Col(gerar_card("Tempo Médio por Perfil e Gênero", fig2_p3), width=6),
            ]),
        ]),

        # 🔵 Aba 3: Evolução
        dbc.Tab(label="Evolução do Tempo de Espera", tab_id="tab-evolucao", children=[
            dbc.Row([
                dbc.Col(gerar_card("Evolução Mensal com COVID", fig1_p2), width=12),
            ]),
            dbc.Row([
                dbc.Col(gerar_card("Evolução Anual", fig2_p2), width=12),
            ]),
        ]),

        # 🔵 Aba 4: Análise Hospital/Ano
        dbc.Tab(label="Hospital x Ano", tab_id="tab-hospital", children=[
            dbc.Row([
                dbc.Col(gerar_card("Tempo Médio - Heatmap Hospital x Ano", fig1_p4), width=12),
            ])
        ]),

        # 🔥 Aba 5: Dados Tabulares
        dbc.Tab(label="Base de Dados", tab_id="tab-dados", children=[
            dbc.Row([
                dbc.Col(html.H5(
                    "Base de dados consolidada utilizada nesta análise:",
                    className="text-center text-secondary my-3"
                ), width=12)
            ]),
            dbc.Row([
                dbc.Col(dash_table.DataTable(
                    data=df_espera.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in df_espera.columns],
                    page_size=20,
                    style_table={'overflowX': 'auto'},
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold',
                        'textAlign': 'center'
                    },
                    style_cell={
                        'textAlign': 'center',
                        'padding': '5px',
                        'minWidth': '100px', 'width': '100px', 'maxWidth': '180px',
                        'whiteSpace': 'normal',
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ],
                    filter_action="native",
                    sort_action="native",
                    sort_mode="multi",
                    column_selectable="single",
                    selected_columns=[],
                    selected_rows=[],
                    page_action="native",
                ), width=12)
            ])
        ]),

    ], id="tabs", active_tab="tab-geral", class_name="mb-4"),

    # ===== Footer =====
    dbc.Row([
        dbc.Col(html.Hr())
    ]),
    dbc.Row([
        dbc.Col(html.P(
            f"Dashboard desenvolvido por {NOMES} • Dados atualizados automaticamente • Versão 1.0",
            className="text-center text-muted"
        ))
    ])

], fluid=True)

In [159]:
if __name__ == '__main__':
    app.run(debug=True)